# 4. Spatial Smoothing, Regionalization, and Neighborhood Analysis
## 4.1 Spatial Smoothing
#### 4.1.1 Introduction
In many studies the target variable may need to modeled using a rate or normalizaed value due to a number of reasons including:
* Varying underlying population
* Variation in the age structure of underlying population
* Variation in environmental variables across a study area
* Arbitrarily shaped areal units

These rates can be heavily influenced by high or low raw counts. In order to reduce the effect of high raw value counts on the rates, spatial smoothing can be used to moderate extreme values at an observation.

#### 4.1.2 Mean and Median Smoothing
One of the simplest ways to conduct spatial smoothing is by using locally weighted smoothing. This technique takes values from surrounding areal units (based on the W matrix) and uses a weighted average of the weights to produce a value at a given observation.

In [1]:
# Conduct mean smoothing on a dataset
import pysal
import numpy as np
from pysal.esda import smoothing as sm
tw = pysal.queen_from_shapefile("./data/stpete_cenacs_2014.shp") # W
w = pysal.open('./data/stpete_cenacs_2014.gal', 'r').read()
stp = pysal.open("./data/stpete_cenacs_2014.dbf", 'r')
e, b = np.array(stp[:,58]), np.array(stp[:,15])
if not w.id_order_set: w.id_order = range(1,len(stp) + 1)
rate = sm.Disk_Smoother(e, b, w)
rate.r

/home/tankofvines/anaconda3/lib/python3.5/site-packages/pysal/esda/smoothing.py:1223: RuntimeWarning: invalid value encountered in true_divide
  r = e * 1.0 / b


array([[ 0.10888572],
       [ 0.06087701],
       [ 0.12075095],
       [ 0.24290032],
       [ 0.0638897 ],
       [ 0.24481339],
       [ 0.21721222],
       [ 0.17797916],
       [ 0.21247315],
       [ 0.14050576],
       [ 0.07901721],
       [ 0.03396642],
       [ 0.05301667],
       [ 0.07772376],
       [ 0.06357303],
       [ 0.14900607],
       [ 0.11448345],
       [ 0.13617885],
       [ 0.07513913],
       [ 0.12679005],
       [ 0.12729641],
       [ 0.122248  ],
       [ 0.09043967],
       [ 0.03851204],
       [ 0.12162695],
       [ 0.09799713],
       [ 0.14118007],
       [ 0.13723431],
       [ 0.12110901],
       [ 0.11112569],
       [ 0.09992835],
       [ 0.10183142],
       [ 0.08847661],
       [ 0.13054532],
       [ 0.13499327],
       [ 0.0729697 ],
       [ 0.13845319],
       [        nan],
       [ 0.16702281],
       [ 0.0827519 ],
       [ 0.12566641],
       [ 0.13385139],
       [ 0.12777731],
       [ 0.11283037],
       [ 0.08902386],
       [ 0

In [2]:
rate = sm.Spatial_Median_Rate(e, b, w)
rate.r

/home/tankofvines/anaconda3/lib/python3.5/site-packages/pysal/esda/smoothing.py:1331: RuntimeWarning: invalid value encountered in true_divide
  self.r = e * 1.0 / b
/home/tankofvines/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


array([ 0.08538212,  0.04533566,  0.07568807,  0.25781951,  0.04533566,
        0.2507091 ,  0.23896104,  0.16282109,  0.18828214,  0.10715929,
        0.07333725,  0.03539685,  0.05381555,  0.05437666,  0.06497175,
        0.15343935,  0.08893428,  0.09810127,  0.06497175,  0.14316125,
        0.09511124,  0.12343865,  0.10339734,  0.03842314,  0.12749022,
        0.13815789,  0.14084507,  0.12299194,  0.08917197,  0.12327292,
        0.10137153,  0.11968085,  0.11361665,  0.11298482,  0.11366493,
        0.07055985,  0.13214186,         nan,  0.18828214,  0.08441657,
        0.10873606,  0.12859712,  0.13824731,  0.12241348,  0.08946208,
        0.10118164,  0.04350116,  0.03620404,  0.02083333,  0.11972201,
               nan,  0.21479714,  0.10915968,  0.25535485,         nan,
        0.12816411,  0.12816411,  0.12443571,  0.09318756,  0.1451767 ,
        0.10344828,  0.07798496,  0.09663866,  0.08690099,  0.07225898,
        0.25535485,  0.1451767 ,  0.05975724,  0.05975724,  0.11

#### 4.1.3 Spatial Missing Value Imputation


## 4.2 Regionalization
#### 4.2.1 Introduction
Spatial regions are groups of observations that share similar qualities to other observations they share a boundary with. Regionalization is a technique used assign regions to a spatial dataset and attribute region membership to the observation. The technique is analagous to unsupervised learning methods such as hierarchical or k-means clustering that determine cluster membership in feature space.
#### 4.2.2 max-p
Max-p is a regionalization method that does not require the number of regions (clusters) apriori. The only required values are a floor constraint for the minimum number of observations within a region. Let's calculate max-p for the median household income values in St. Petersburg, FL.

In [5]:
# Calculate max-p for a set of areal units
stnorm = pysal.open("./data/stpete_cenacs_2014_norms.dbf", 'r')
y = np.array(stnorm)
r = pysal.Maxp(w, y, floor = 5, floor_variable = np.ones((231, 1)), initial = 99)
r.regions

[['39', '105', '64', '121', '62', '186', '182'],
 ['145', '90', '146', '23', '28', '35', '79', '46', '147', '11', '86', '48'],
 ['47', '148', '123', '1', '125', '77', '88', '165', '59', '133', '63', '91'],
 ['172', '215', '155', '214', '225'],
 ['200', '49', '193', '196', '3'],
 ['131', '189', '71', '31', '117'],
 ['113', '37', '38', '205', '50', '107'],
 ['44', '45', '43', '194', '192', '140', '98', '132', '106'],
 ['2', '144', '190', '52', '149'],
 ['87', '89', '94', '96', '93'],
 ['99', '16', '14', '97', '12'],
 ['27', '219', '60', '9', '58'],
 ['68', '72', '67', '73', '75', '136'],
 ['227', '61', '83', '102', '84'],
 ['198', '54', '203', '211', '204'],
 ['20', '108', '34', '82', '25', '17', '81', '122', '195'],
 ['158', '224', '159', '134', '201'],
 ['51', '6', '181', '212', '80'],
 ['66', '175', '177', '230', '151', '226'],
 ['120', '70', '119', '124', '0'],
 ['222', '223', '130', '138', '26', '173', '163', '110', '167'],
 ['76', '4', '114', '116', '69'],
 ['53', '65', '168', '166

## 4.3 Neighborhood Analysis